In [2]:
import json
import requests
import pandas as pd
import boto3
#import pyarrow
from fractions import Fraction

In [35]:
# --------------------------------------------------- CÓDIGO DE LAMBDA LAUREATES --------------------------------------------

url = 'http://api.nobelprize.org/2.1/laureates?ID={}'
    
#TODO Pasar parametro por API
laureate_id = 467
    
info_laureate = requests.get(url.format(laureate_id))
    
resultApi=info_laureate.json()
    
laureateDict=resultApi['laureates'][0]

if 'gender' in laureateDict:   
    fullName=laureateDict['fullName']['en']
    fileName=laureateDict['fileName']
    gender=laureateDict['gender']
    birth=laureateDict['birth']['date']
else:
    fullName='None'
    fileName=laureateDict['fileName']
    gender='None'
    birth='None'
    
dfParquet=pd.DataFrame(columns=['id','fullName','fileName','gender','birth'])
    
dfParquet=dfParquet.append({
    
    'id':laureateDict['id'],
    'fullName': fullName,
    'fileName':fileName,
    'gender':gender,
    'birth':birth
    
} ,ignore_index=True)
     

s3=boto3.client('s3')
    
content=dfParquet.to_parquet()
prueba=dfParquet.to_json()
s3.put_object(Body=content, Bucket='alvaro-munoz-nobelprice', Key='raw/laureate/'+ str(laureate_id)  +'.parquet')

C:\Users\steve\AppData\Local\Temp\ipykernel_13412\1689382244.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfParquet=dfParquet.append({


{'ResponseMetadata': {'RequestId': '9MVF0Y16DF3Q5EKH',
  'HostId': 'QwBrTQQ6hQPtYkbaUtzwQny6Ga8msrdDMmDSS1T5R4KJ5sYh7zUf5mHbHCINKj1AGeEP+ocf+GE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'QwBrTQQ6hQPtYkbaUtzwQny6Ga8msrdDMmDSS1T5R4KJ5sYh7zUf5mHbHCINKj1AGeEP+ocf+GE=',
   'x-amz-request-id': '9MVF0Y16DF3Q5EKH',
   'date': 'Mon, 08 Aug 2022 12:11:17 GMT',
   'etag': '"319594dece2ff297bb812061052f7bd2"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"319594dece2ff297bb812061052f7bd2"'}

In [ ]:
url = 'http://api.nobelprize.org/2.1/laureates?ID={}'
    
#TODO Pasar parametro por API
laureate_id = 467
    
info_laureate = requests.get(url.format(laureate_id))

In [13]:
# --------------------------------------------------- CÓDIGO DE LAMBDA NOBEL PRIZES --------------------------------------------

event_category = "che"
event_year = 2019
url = 'http://api.nobelprize.org/2.1/nobelPrize/{}/{}' 
    
resultApi = requests.get(url.format(event_category,event_year))

laureates_id=[]
laureates_portion=[]
resultApi=resultApi.json()
resultDict=resultApi[0]

try:
    prueba2=resultDict['laureates']
    dateAwarded=resultDict['dateAwarded']
    for i in range(len(prueba2)):
        laureates_id.append(prueba2[i]['id'])
        laureates_portion.append(prueba2[i]['portion'])
except:
    laureates_id=None
    dateAwarded=None
    laureates_portion=None
        
dfNobelPrize = pd.DataFrame(columns=['awardYear','category','categoryFullName','dateAwarded','prizeAmount','prizeAmountAdjusted','laureates_id','portion'] )
    
    
dfNobelPrize=dfNobelPrize.append({
    
                'id': laureates_id,
                'portion': laureates_portion,
                'awardYear': resultDict['awardYear'],
                'category': resultDict['category']['en'],
                'categoryFullName': resultDict['categoryFullName']['en'],
                'dateAwarded': dateAwarded,
                'prizeAmount': resultDict['prizeAmount'],
                'prizeAmountAdjusted': resultDict['prizeAmountAdjusted']
                    
        }, ignore_index=True)
    
dfNobelPrize=dfNobelPrize.set_index('id')
    
dfNobelPrize.head(60)
    
s3=boto3.client('s3')
content=dfNobelPrize.to_parquet('temp.parquet')
s3.put_object(Body=dfNobelPrize.to_parquet(), Bucket='alvaro-munoz-nobelprice', Key='raw/nobelPrize/'+ event_category +'-'+ str(event_year)  +'.parquet')

TypeError: to_parquet() missing 1 required positional argument: 'path'

In [2]:
# ------------------------ LECTURA DESDE EL S3 -------------------------------

# df_nobel_s3 = pd.read_parquet(path='s3://alvaro-munoz-nobelprice/raw/nobelPrize/')
df_laureates_s3 = pd.read_parquet(path='s3://cidaen/raw/laureate/')

df_laureates_s3.head(60)
# df_nobel_s3.head(60)

FileNotFoundError: cidaen/raw/laureate/

In [7]:
#  ------------------------------- ETL BRONZE LAMBDA ----------------------------------
import json
import pandas as pd
import boto3

df_laureates=pd.read_parquet("s3://alvaro-munoz-nobelprice/raw/laureate")
df_nobel_prizes=pd.read_parquet("s3://alvaro-munoz-nobelprice/raw/nobelPrize")


new_types={
        "id":"int",
        "fullName":"string",
        "fileName":"string",
        "gender":"string",
        "birth":"string"
    }
        
    
df_laureates=df_laureates.astype(new_types)
df_laureates.dropna(subset=['fullName'],inplace=True)
df_laureates=df_laureates.drop_duplicates(subset=['id','fullName','fileName'])
df_laureates.sort_index(inplace=True)
        
        
    
df_nobel_prizes.dropna(subset = ['dateAwarded'], inplace = True)
df_nobel_prizes=df_nobel_prizes.explode('laureates_id',ignore_index=True)
df_nobel_prizes=df_nobel_prizes.explode('portion')
        
new_types = {
    'awardYear':'string',
    'category':'string',
    'categoryFullName':'string',
    'dateAwarded':'string',
    'prizeAmount':'int',
    'prizeAmountAdjusted':'int',
    'laureates_id':'int',
}
        
df_nobel_prizes=df_nobel_prizes.astype(new_types)

mix_df = df_laureates.join(df_nobel_prizes, on='id', how='inner')
df_bronze_laureates=mix_df.rename(columns={'fileName':'file_name'})

mix_df.head(60)

# s3=boto3.client('s3')
        
# content=df_bronze_laureates.to_parquet()
# prueba=df_bronze_laureates.to_json()


# s3.put_object(Body=content, Bucket='alvaro-munoz-nobelprice', Key='raw/bronze/bronze_laureates.parquet')

,id,fullName,fileName,gender,birth,awardYear,category,categoryFullName,dateAwarded,prizeAmount,prizeAmountAdjusted,laureates_id,portion
0,2,Hendrik Antoon Lorentz,lorentz,male,1853-07-18,2010,Chemistry,The Nobel Prize in Chemistry,2010-10-06,10000000,11015580,853,1/3
0,2,Hendrik Antoon Lorentz,lorentz,male,1853-07-18,2010,Chemistry,The Nobel Prize in Chemistry,2010-10-06,10000000,11015580,853,1/3
0,2,Hendrik Antoon Lorentz,lorentz,male,1853-07-18,2010,Chemistry,The Nobel Prize in Chemistry,2010-10-06,10000000,11015580,853,1/3
4,5,Pierre Curie,pierre-curie,male,1859-05-15,2018,Chemistry,The Nobel Prize in Chemistry,2018-10-03,9000000,9163200,965,1/2
4,5,Pierre Curie,pierre-curie,male,1859-05-15,2018,Chemistry,The Nobel Prize in Chemistry,2018-10-03,9000000,9163200,965,1/4
4,5,Pierre Curie,pierre-curie,male,1859-05-15,2018,Chemistry,The Nobel Prize in Chemistry,2018-10-03,9000000,9163200,965,1/4
5,6,"Marie Curie, née Sklodowska",marie-curie,female,1867-11-07,2005,Literature,The Nobel Prize in Literature,2005-10-13,10000000,11923798,801,1


In [9]:
#  ------------------------------- LAMBDA SILVER ----------------------------------

import json
import pandas as pd
import boto3
import numpy as np
import fractions


    
df_bronze_laureates = pd.read_parquet("s3://alvaro-munoz-nobelprice/raw/bronze")
    
#Rename of the columns
df_bronze_laureates = df_bronze_laureates.rename(columns={'prizeAmount':'prize_amount',
                                                            'awardYear':'year', 
                                                            'laureates_id' : 'laureate_id', 
                                                            'portion' : 'laureate_portion', 
                                                            'categoryFullName' : 'category_full_name', 
                                                            'dateAwarded' : 'date_awarded', 
                                                            'prizeAmountAdjusted' : 'prize_amount_adjusted', 
                                                            'fullName' : 'full_name'})
df_bronze_laureates['laureate_portion'] = fractions.Fraction(df_bronze_laureates['laureate_portion'])
#Change "laureates_portion" into a float column
refactor_columns = {
    'laureate_portion' : 'float'
}
    
df_bronze_laureates = df_bronze_laureates.astype(refactor_columns)
    
#Creation of the column: prize_amount_real
prize_amount_real_col = pd.DataFrame(columns=['prize_real_amount'] )
prize_amount_real_col['prize_real_amount'] = np.sqrt(df_bronze_laureates['prize_amount'] * df_bronze_laureates['laureate_portion'])
    
#Adding the new column
df_bronze_new = df_bronze_laureates.assign(prize_real_amount = prize_amount_real_col)
    
    
#Creating the file "silver_laureates.parquet" in the bucket S3
s3=boto3.client('s3')
    
content=df_bronze_new.to_parquet()
prueba=df_bronze_new.to_json()
s3.put_object(Body=content, Bucket='cidaen', Key='raw/silver/silver_laureates.parquet')

ValueError: DataFrame index must be unique for orient='columns'.

In [2]:
# df_laureates_s3 = pd.read_parquet('s3://alvaro-munoz-nobelprice/raw/silver')

df_laureates_s3 = pd.read_parquet("s3://alvaro-munoz-nobelprice/raw/bronze")
df_laureates_s3.head(60)

,awardYear,category,categoryFullName,dateAwarded,prizeAmount,prizeAmountAdjusted,laureates_id,portion,id,fullName,file_name,gender,birth
